In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Reviews.csv')

In [81]:
u_userid = df['UserId'].unique()
u_productid = df['ProductId'].unique()

In [79]:
u_userid

array(['A3SGXH7AUHU8GW', 'A1D87F6ZCVE5NK', 'ABXLMWJIXXAIN', ...,
       'A28KG5XORO54AY', 'A121AA1GQV751Z', 'A3LGQPJCZVL9UC'], dtype=object)

In [82]:
u_productid

array(['B006Q820X0', 'B008I1XPKA', 'B0026LJ3EA', ..., 'B002SZZ00U',
       'B0029XEXZE', 'B0029XITW2'], dtype=object)

In [85]:
productid_index = {}
for index,productid in enumerate(u_productid):
    productid_index[productid] = index

In [40]:
df['UserId_ProductId'] = df.apply(lambda x:'%s_%s' % (x['UserId'],x['ProductId']),axis=1)

In [65]:
df.sort_values(by=['UserId_ProductId'],inplace=True)

In [66]:
df.reset_index(inplace=True)

In [ ]:
with open('test.csv','w') as f:
    header = ['UserID,ProductId']
    dictionary = {'UserId':u_productid}
    data_bak = [0]*len(u_productid)
    writer = csv.writer(f)
    for i in range(len(df)):
        row = df.loc[i:i]
        userid = row.iloc[0]['UserId']
        productid = row.iloc[0]['ProductId']
        score = row.iloc[0]['Score']
        if userid not in dictionary:
            for key,value in dictionary.items():
                data = key
                for v in value:
                    data += ','+str(v)
            f.write(data+'\n')
            dictionary = {}
            dictionary[userid] = data_bak
            dictionary[userid][productid_index[productid]] = score
        else:
            dictionary[userid][productid_index[productid]] = score


## preprocessing - (to3tuple)

In [56]:
import pandas as pd
import numpy as np
df = pd.read_csv('Reviews.csv')
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [57]:
#remove users appears once
df = df[df.groupby('UserId').UserId.transform(len) > 1]

In [58]:
#moderate user scores based on user input and group input
df['AvgUserScore'] = df['Score'].groupby(df['UserId']).transform('mean')
df['AvgProductScore'] = df['Score'].groupby(df['ProductId']).transform('mean')
df['ModeratedScore'] = 0.75*df['AvgUserScore'] + 0.25*df['AvgProductScore']

C:\Users\qq393\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\qq393\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\qq393\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [59]:
users = df['UserId'].unique()
items = df['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df.apply(lambda x: items_dict[x['ProductId']], axis=1)

In [62]:
#ratings = pd.concat([intUserId,intItemId,df['Score']], axis=1)
ratings = pd.concat([intUserId,intItemId,df['ModeratedScore']], axis=1)

In [63]:
ratings.to_csv('Ratings.csv', index=False, header=False)